In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

! pip install rs-datasets

In [3]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

import logging
from replay.data_preparator import DataPreparator, Indexer
from replay.session_handler import State
from rs_datasets import MovieLens

spark = State().session
spark.sparkContext.setLogLevel('ERROR')
logger = logging.getLogger("replay")
spark

In [4]:
data = MovieLens("100k")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116



users


,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067



items


,item_id,title,release_date,imdb_url,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,False,True,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [5]:
preparator = DataPreparator()

indexer = Indexer(user_col='user_id', item_col='item_id')

log = preparator.transform( columns_mapping={'user_id': 'user_id',
                                             'item_id': 'item_id',
                                             'relevance': 'rating',
                                             'timestamp': 'timestamp'
                                            }, 
                            data=data.ratings)

indexer.fit(users=log.select('user_id'),
            items=log.select('item_id'))

log = indexer.transform(df=log)
log.show(3)

user_features = preparator.transform(columns_mapping={'user_id': 'user_id'},
                                     data=data.users)
user_features = indexer.transform(df=user_features)
user_features.show(3)

item_features = preparator.transform(columns_mapping={'item_id': 'item_id'}, 
                                     data=data.items)
item_features = indexer.transform(df=item_features)
item_features.show(3)

11-Jul-23 14:25:22, replay, INFO: Columns with ids of users or items are present in mapping. The dataframe will be treated as an interactions log.
11-Jul-23 14:25:32, replay, INFO: Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.


+--------+--------+---------+-------------------+
|user_idx|item_idx|relevance|          timestamp|
+--------+--------+---------+-------------------+
|     645|     287|      3.0|1997-12-04 18:55:49|
|     382|      37|      3.0|1998-04-04 23:22:22|
|     282|    1059|      1.0|1997-11-07 10:18:36|
+--------+--------+---------+-------------------+
only showing top 3 rows



11-Jul-23 14:25:33, replay, INFO: Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.


+--------+------+---+----------+--------+
|user_idx|gender|age|occupation|zip_code|
+--------+------+---+----------+--------+
|      76|    24|  M|technician|   85711|
|     485|    53|  F|     other|   94043|
|     535|    23|  M|    writer|   32067|
+--------+------+---+----------+--------+
only showing top 3 rows

+--------+-----------------+------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+-----+-------+
|item_idx|            title|release_date|            imdb_url|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|  War|Western|
+--------+-----------------+------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+-----+-------+
|       7| Toy Story (19

In [6]:
# Make embeddings
pd_items = item_features.orderBy("item_idx").drop("item_idx","title", "release_date", "imdb_url").toPandas().astype(int)

In [7]:
pd_items

,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
3,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1679,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0


In [8]:
np_items = pd_items.to_numpy()

In [12]:
np_items[1681]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
# Clusterize
kmeans = KMeans(n_clusters=2)
kmeans.fit(np_items)

/home/arqa/MyRePlay/.venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=2)

In [14]:
cluster_map = kmeans.predict(np_items)

In [15]:
cl_0 = np.where(cluster_map == 0)
cl_1 = np.where(cluster_map == 1)

In [16]:
np_items[cl_0].shape

(957, 19)

In [33]:
dom_mask = np.full(np_items.shape[0], True)

In [34]:
dom_mask.size

1682

In [35]:
np_items[dom_mask][cluster_map == 1].shape

(725, 19)

In [39]:
dom_mask_cl_0 = np.full(dom_mask.size, False)
dom_mask_cl_0[dom_mask] = (cluster_map == 0)
dom_mask_cl_1 = np.full(dom_mask.size, False)
dom_mask_cl_1[dom_mask] = (cluster_map == 1)

In [40]:
dom_mask_cl_0[dom_mask_cl_0 == True].shape

(957,)

In [41]:
dom_mask_cl_1[dom_mask_cl_1 == True].shape

(725,)

In [152]:
kmeans.fit(np_items[dom_mask_cl_0])
cluster_map = kmeans.predict(np_items[dom_mask_cl_0])

/home/arqa/MyRePlay/.venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [156]:
dom_mask_cl_0_0 = np.full(dom_mask_cl_0.shape[0], False)
dom_mask_cl_0_0[dom_mask_cl_0] = (cluster_map == 0)
dom_mask_cl_0_1 = np.full(dom_mask_cl_0.shape[0], False)
dom_mask_cl_0_1[dom_mask_cl_0] = (cluster_map == 1)

In [158]:
dom_mask_cl_0_0[dom_mask_cl_0_0== True].shape

(626,)

In [159]:
dom_mask_cl_0_1[dom_mask_cl_0_1 == True].shape

(99,)

In [137]:
dom_mask

array([False,  True,  True, ..., False,  True, False])

In [ ]:
dom_mask_cl_0

In [ ]:
class Node:
    """Node of an item tree"""

    def __init__(
        self,
        tree,
        level,
        parent,
        dom_mask,
    ):
        """
        :param level: level of the node in the tree
        :param parent: the parent node
        :dom_mask: boolean mask of size equal to the total number of items that 
        represents the node's item domain (a set of items reachable from this 
        node)
        """
        self.tree = tree
        self.level = level
        self.parent = parent
        self.dom_mask = dom_mask
    
    def procreate(self):
        if (level < tree.depth)

    def fit_forward(self):
    
    def fit_backward(self):
    
    def predict(self):
        
            

In [42]:
class ItemTree:
    """An item tree class"""

    def __init__(
            self, 
            items, 
            depth, 
            #arity, 
            clusterer,
    ):
        """
        :param items: item embeddings
        :param depth: depth of the item tree
        #:param arity: number of children of each node
        """
        self.items = items
        self.depth = depth
        #self.arity = arity
        self.clusterer = clusterer
        
        self.root = Node(   tree=self,
                            level=0, 
                            parent=None, 
                            dom_mask=np.full(items.shape[0], True)
                        )
        
        self.root.procreate()


class Node:
    """Node of an item tree"""

    def __init__(self, tree, level, parent, dom_mask):
        """
        :param level: level of the node in the tree
        :param parent: the parent node
        :dom_mask: boolean mask of size equal to the total number of items that 
        represents the node's item domain (a set of items reachable from this 
        node)
        """
        self.tree = tree
        self.level = level
        self.parent = parent
        self.dom_mask = dom_mask
        self.children = []
    
    def procreate(self):
        if (self.level < self.tree.depth):
            cluster_map = self.tree.clusterer.fit_predict(
                                                self.tree.items[self.dom_mask]
                                                )
            n_children = self.tree.clusterer.cluster_centers_.shape[0]
            for k in range(n_children):
            #for k in range(self.tree.arity):
                dom_mask_k = np.full(self.dom_mask.size, False)
                dom_mask_k[self.dom_mask] = (cluster_map == k)

                child = Node(   tree=self.tree,
                                level=(self.level + 1), 
                                parent=self, 
                                dom_mask=dom_mask_k
                            )
                child.procreate()
                self.children.append(child)
        else:
            self.children = None

    def fit_forward(self):
        0
    
    def fit_backward(self):
        0
    
    def predict(self):
        0
        

In [43]:
kmeans = KMeans(n_clusters=3)
item_tree = ItemTree(items=np_items, depth=2, clusterer=kmeans)

/home/arqa/MyRePlay/.venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/arqa/MyRePlay/.venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/arqa/MyRePlay/.venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/arqa/MyRePlay/.venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [54]:
print(np_items[item_tree.root.dom_mask].shape)

(1682, 19)


In [55]:
print(np_items[item_tree.root.children[0].dom_mask].shape)
print(np_items[item_tree.root.children[1].dom_mask].shape)
print(np_items[item_tree.root.children[2].dom_mask].shape)

(502, 19)
(544, 19)
(636, 19)


In [59]:
print(np_items[item_tree.root.children[0].children[0].dom_mask].shape)
print(np_items[item_tree.root.children[0].children[1].dom_mask].shape)
print(np_items[item_tree.root.children[0].children[2].dom_mask].shape)
print(np_items[item_tree.root.children[1].children[0].dom_mask].shape)
print(np_items[item_tree.root.children[1].children[1].dom_mask].shape)
print(np_items[item_tree.root.children[1].children[2].dom_mask].shape)
print(np_items[item_tree.root.children[2].children[0].dom_mask].shape)
print(np_items[item_tree.root.children[2].children[1].dom_mask].shape)
print(np_items[item_tree.root.children[2].children[2].dom_mask].shape)

(327, 19)
(97, 19)
(78, 19)
(105, 19)
(179, 19)
(260, 19)
(493, 19)
(85, 19)
(58, 19)


In [73]:
print(item_tree.root.children[2].children[0].children)

None
